In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 39.2 MB/s eta 0:00:00


In [ ]:
!pip -q install -U "spacy==3.7.4"
!python -m spacy download es_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.0/865.0 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.44.1 requires typer<1.0,>=0.12; sys_platform != "emscripten", but you have typer 0.9.4 which is incompatible

In [ ]:
import pandas as pd
import json
import fitz
import re
import unicodedata
import spacy


# 1. Extracción y Preparación del Corpus:

## 1.1. Extraer las entrevistas

In [ ]:
# Cargar datos de los testimonios
with open('/content/entrevistas_all_2023-03-21_14-24_05.json', 'r') as file:
    data = json.load(file)
df_testimonios = pd.DataFrame(data)
df_testimonios.head(3)

,id_doc,pages,text
0,5eb4e7bfda6da3b502da3589.pdf,30,001-VI-00008\nTEST: Bajo la apariencia de una...
1,5eb4e7c0da6da3b502da358e.pdf,30,001-VI-00008\nTEST: Bajo la apariencia de una...
2,5eb4e7c1da6da3b502da3593.pdf,15,"ENT:\nBueno, hoy es --------------------------..."


In [ ]:
df_testimonios.shape

(2486, 3)

In [ ]:
# 1. Eliminar vacíos
vacios = df_testimonios["text"].eq("   ")
print(f"Entrevistas vacías detectadas: {vacios.sum()}")
df_testimonios = df_testimonios.loc[~vacios].reset_index(drop=True)

Entrevistas vacías detectadas: 2


In [ ]:
# Contar duplicados en la columna 'text'
duplicados = df_testimonios["text"].duplicated().sum()
print(f"Se encontraron {duplicados} entrevistas duplicadas.")

# Eliminar duplicados
df_testimonios = df_testimonios.drop_duplicates(subset="text").reset_index(drop=True)


Se encontraron 268 entrevistas duplicadas.


In [ ]:
df_testimonios.shape

(2216, 3)

## 1.2. Limpieza preliminar de las entrevistas

In [ ]:
def limpiar_texto(texto):

    # Normalizar unicode (acentos, espacios raros, símbolos)
    texto = unicodedata.normalize("NFKC", texto)

    texto = texto.lower()  # normalizamos todo a minúsculas

    # Eliminar encabezados tipo 001-VI-00008
    texto = re.sub(r"\b\d{2,5}-[a-z]{2,}-\d{3,}\b", " ", texto)

    # Eliminar paréntesis con números tipo _(868)
    texto = re.sub(r"_\(\d+\)", " ", texto)

    # Eliminar marcas de tiempo tipo 01:30:51 o 00:08–00:48
    texto = re.sub(r"\d{1,2}:\d{2}(:\d{2})?", " ", texto)
    texto = re.sub(r"\d{1,2}:\d{2}[–-]\d{1,2}:\d{2}", " ", texto)

    # Buscar intervenciones de TEST/TEST1/TEST 2... hasta ENT
    fragmentos = re.findall(r"test\s*\d*:(.*?)(?=ent\s*\d*:|ent:|$)", texto, flags=re.S | re.I)

    if fragmentos:
        texto = " ".join(fragmentos)
    else:
        # fallback: usar texto completo (ya en minúscula)
        texto = texto

    # Eliminar etiquetas entre corchetes [....]
    texto = re.sub(r"\[.*?\]", " ", texto)

    # Eliminar guiones largos
    texto = re.sub(r"-{2,}", " ", texto)

    # Quitar números pegados a palabras tipo "familiar1" → "familiar"
    texto = re.sub(r"\b([a-záéíóúñü]+)\d+\b", r"\1", texto)

    # Separar números de palabras pegadas: 2personas → 2 personas
    texto = re.sub(r"(\d+)([a-záéíóúñü]+)", r"\1 \2", texto)

    # Quitar caracteres no alfanuméricos
    texto = re.sub(r"[^a-záéíóúñü0-9\s]", " ", texto)

    # Normalizar espacios
    texto = re.sub(r"\s+", " ", texto).strip()

    return texto

# Aplicar al dataframe
df_testimonios["text_clean"] = df_testimonios["text"].apply(limpiar_texto)

## 1.3. Tokenizacion y eliminacion de stopwords

In [ ]:
# Tokenizacion y eliminacion de stopwords

# Cargar modelo de spaCy en español
nlp = spacy.load("es_core_news_sm")

def tokenizar(texto):
    doc = nlp(texto)
    # Tokens con stopwords (y sin puntuación)
    tokens_all = [tok.text for tok in doc if not tok.is_punct]
    # Tokens sin stopwords ni puntuación
    tokens_no_stop = [tok.text for tok in doc if not tok.is_stop and not tok.is_punct]
    return tokens_all, tokens_no_stop

# Aplicar al corpus
df_testimonios[["tokens_all", "tokens_no_stop"]] = df_testimonios["text_clean"].apply(
    lambda x: pd.Series(tokenizar(x))
)

# Ejemplo
print(df_testimonios[["text_clean", "tokens_all", "tokens_no_stop"]].head(1).to_string())


## 1.4. Lematizacion

In [ ]:
from google.colab import files

def lematizar(texto):
    doc = nlp(texto)
    # Lemas con stopwords (sin puntuación)
    lemmas_all = [tok.lemma_ for tok in doc if not tok.is_punct]
    # Lemas sin stopwords ni puntuación
    lemmas_no_stop = [tok.lemma_ for tok in doc if not tok.is_stop and not tok.is_punct]
    return lemmas_all, lemmas_no_stop

# Aplicar al corpus
df_testimonios[["lemmas_all", "lemmas_no_stop"]] = df_testimonios["text_clean"].apply(
    lambda x: pd.Series(lematizar(x))
)

# Ejemplo
print(df_testimonios[["tokens_all", "tokens_no_stop", "lemmas_all", "lemmas_no_stop"]].head(1).to_string())

# Guardar a CSV
output_file = "testimonios_procesados.csv"
df_testimonios.to_csv(output_file, index=False, encoding="utf-8")

# Descargar automáticamente al PC
files.download(output_file)


                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   tokens_all                                                                                                                                                                                                                           

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 1.5. Limpieza post

In [ ]:
# Función para limpiar tokens o lemmas (listas)
def limpiar_tokens(lista):
    if not isinstance(lista, list):
        return []
    return [tok for tok in lista if not re.fullmatch(r"(test|ent)\d*", tok)]

# Función para limpiar texto plano
def limpiar_texto_extra(texto):
    if not isinstance(texto, str):
        return ""
    return re.sub(r"\b(test|ent)\d*\b", "", texto).strip()

# Aplicar sobre tokens y lemmas
df_testimonios["tokens_all"]     = df_testimonios["tokens_all"].apply(limpiar_tokens)
df_testimonios["tokens_no_stop"] = df_testimonios["tokens_no_stop"].apply(limpiar_tokens)
df_testimonios["lemmas_all"]     = df_testimonios["lemmas_all"].apply(limpiar_tokens)
df_testimonios["lemmas_no_stop"] = df_testimonios["lemmas_no_stop"].apply(limpiar_tokens)

# Aplicar sobre texto plano
df_testimonios["text_clean"] = df_testimonios["text_clean"].apply(limpiar_texto_extra)

# Normalizar espacios otra vez (pueden quedar dobles tras quitar palabras)
df_testimonios["text_clean"] = df_testimonios["text_clean"].str.replace(r"\s+", " ", regex=True).str.strip()


In [ ]:
df_testimonios

,id_doc,pages,text,text_clean,tokens_all,tokens_no_stop,lemmas_all,lemmas_no_stop
0,5eb4e7bfda6da3b502da3589.pdf,30,001-VI-00008\nTEST: Bajo la apariencia de una...,bajo la apariencia de una democracia formal en...,"[bajo, la, apariencia, de, una, democracia, fo...","[apariencia, democracia, formal, colombia, viv...","[bajo, el, apariencia, de, uno, democracia, fo...","[apariencia, democracia, formal, colombia, viv..."
1,5eb4e7c1da6da3b502da3593.pdf,15,"ENT:\nBueno, hoy es --------------------------...",sí con mucho gusto mi nombre es nací en en est...,"[sí, con, mucho, gusto, mi, nombre, es, nací, ...","[gusto, nombre, nací, estudié, colegios, colom...","[él, con, mucho, gusto, mi, nombre, ser, nací,...","[gusto, nombre, nací, estudié, colegio, colomb..."
2,5eb4e7c1da6da3b502da3595.pdf,38,001-VI-00011\n \n001-VI-00011_(48057): 01:30:...,buenas tardes gracias con mucho gusto con much...,"[buenas, tardes, gracias, con, mucho, gusto, c...","[tardes, gracias, gusto, gusto, nombre, inicio...","[buena, tarde, gracias, con, mucho, gusto, con...","[tarde, gracias, gusto, gusto, nombre, inicio,..."
3,5eb4e7c1da6da3b502da3599.pdf,24,ENT: Pues buenas tardes TEST Le damos la bienv...,gracias entonces bueno buenas tardes muchas gr...,"[gracias, entonces, bueno, buenas, tardes, muc...","[gracias, tardes, gracias, espacio, llamo, pap...","[gracias, entonces, bueno, buena, tarde, mucho...","[gracias, tarde, gracia, espacio, llamar, papá..."
4,5eb4e7c2da6da3b502da359d.pdf,17,"AUDIO 001-VI-00015_ (48157)\nENT: Bueno, pues ...",ok buenos días y muchísimas gracias por el esp...,"[ok, buenos, días, y, muchísimas, gracias, por...","[ok, muchísimas, gracias, espacio, ok, llego, ...","[ok, buen, día, y, muchísimar, gracia, por, el...","[ok, muchísimar, gracia, espacio, ok, llego, r..."
...,...,...,...,...,...,...,...,...
2211,625f3e67755cba02e57a5ee7.pdf,41,ENT: Con estocomienzo la entrevista a TEST Mi ...,bueno mi nombre es como usted lo ha dicho naci...,"[bueno, mi, nombre, es, como, usted, lo, ha, d...","[nombre, nacida, arauca, hija, familar, 1, fam...","[bueno, mi, nombre, ser, como, usted, él, habe...","[nombre, nacido, arauca, hija, familar, 1, fam..."
2212,625fd7b8755cba02e5b68d8f.pdf,19,ENT: Bueno. -------días don TEST… TEST [INTERR...,judiciales judiciales si bueno eehm yo empiezo...,"[judiciales, judiciales, si, bueno, eehm, yo, ...","[judiciales, judiciales, eehm, empiezo, acá, t...","[judicial, judicial, si, bueno, eehm, yo, empi...","[judicial, judicial, eehm, empiezo, acá, traba..."
2213,62625339755cba02e5158c7a.pdf,13,"ENT: Bueno TEST, mi corazón,entonces ahora vam...",bueno luego qué pasa que eso me dañó la vida t...,"[bueno, luego, qué, pasa, que, eso, me, dañó, ...","[pasa, dañó, vida, amiga, conocida, compañera,...","[bueno, luego, qué, pasar, que, ese, yo, dañar...","[pasar, dañar, vida, amiga, conocer, compañera..."
2214,6268621d8293e58b8f480f09.pdf,49,077-VI-00013_ (34120)\nENT: --- de --------- d...,pues yo considero que la idea la idea es habla...,"[pues, yo, considero, que, la, idea, la, idea,...","[considero, idea, idea, hablar, sepa, relación...","[pues, yo, considerar, que, el, idea, el, idea...","[considerar, idea, idea, hablar, separ, relaci..."


In [ ]:
# Guardar a CSV
output_file = "entrevistas_procesadas.csv"
df_testimonios.to_csv(output_file, index=False, encoding="utf-8")